### 1) Compare additional topic modelling with TF-IDF (Term Frequency-Inverse Document Frequency) and dimensionality reduction with NMF (Non-negative Matrix Factorization)
### 2) Visualize intertopic distance
### 3) Sum top 100 word tokens and convert to csv for D3 wordcloud visualization

In [7]:
import pickle
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

In [2]:
with open('corpus_final.pkl', 'rb') as picklefile:
    df = pickle.load(picklefile)

In [3]:
df = df.sort_values(by='date')

In [15]:
n_samples = 2000
n_features = 1000
n_topics = 25
n_top_words = 20


def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()


# Use tf-idf features for NMF.
print("Extracting tf-idf features for NMF...")
tfidf_vectorizer = TfidfVectorizer(max_df=0.65, min_df=.01,
                                   max_features=n_features,
                                   stop_words='english')

tfidf = tfidf_vectorizer.fit_transform(df.corpus)


# Use tf (raw term count) features for LDA.
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(max_df=0.65, min_df=.01,
                                max_features=n_features,
                                stop_words='english')

tf = tf_vectorizer.fit_transform(df.corpus)


# Fit the NMF model

nmf = NMF(n_components=n_topics, random_state=1,
          alpha=.1, l1_ratio=.5).fit(tfidf)


print("\nTopics in NMF model:")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)

lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)

lda.fit(tf)

print("\nTopics in LDA model:")
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

Extracting tf-idf features for NMF...
Extracting tf features for LDA...

Topics in NMF model:
Topic #0:
love feel story ask man great guy old experience write walk person hand believe friend leave end turn moment god
Topic #1:
woman men girl sex gender female violence male man young daughter mother story boy black mom voice sister medium father
Topic #2:
sound voice noise listen hear song sing play record wave color visual baby bird note light eye speech affect example
Topic #3:
cancer patient disease doctor health drug treatment medical care surgery hospital medicine heart treat blood die trial test hiv body
Topic #4:
ocean water fish sea animal ice specie planet surface earth deep area island foot creature ship place plastic blue land
Topic #5:
music play song piece sing video end dance hear note audience okay brother sorry dream excite century lady eye ted
Topic #6:
kid school teacher student education teach learn class college math girl high graduate parent community university pro

In [17]:
# from sklearn.externals import joblib
# joblib.dump(nmf, 'nmf_model_1_25_topics_65_df.pkl') 

['nmf_model_1_25_topics_65_df.pkl']

### Visualize NMF topics. NMF produces a greater range of intertopic distance/variation than previous LDA with count-vectorizer. This will be the final model. 

In [58]:
import pyLDAvis, pyLDAvis.sklearn
from IPython.display import display

# Setup to run in Jupyter notebook
pyLDAvis.enable_notebook()

# Create the visualization
vis = pyLDAvis.sklearn.prepare(nmf, tf, tfidf_vectorizer)

# Export as a standalone HTML web page
pyLDAvis.save_html(vis, 'nfm_65_df.html')

# Let's view it!
display(vis)

/Users/brianturner/anaconda/lib/python3.6/site-packages/pyLDAvis/_prepare.py:223: RuntimeWarning: divide by zero encountered in log
  kernel = (topic_given_term * np.log((topic_given_term.T / topic_proportion).T))
/Users/brianturner/anaconda/lib/python3.6/site-packages/pyLDAvis/_prepare.py:240: RuntimeWarning: divide by zero encountered in log
  log_lift = np.log(topic_term_dists / term_proportion)
/Users/brianturner/anaconda/lib/python3.6/site-packages/pyLDAvis/_prepare.py:241: RuntimeWarning: divide by zero encountered in log
  log_ttd = np.log(topic_term_dists)
/Users/brianturner/anaconda/lib/python3.6/site-packages/pyLDAvis/_prepare.py:208: RuntimeWarning: invalid value encountered in multiply
  relevance = lambda_ * log_ttd + (1 - lambda_) * log_lift
/Users/brianturner/anaconda/lib/python3.6/site-packages/pyLDAvis/_prepare.py:208: RuntimeWarning: invalid value encountered in multiply
  relevance = lambda_ * log_ttd + (1 - lambda_) * log_lift


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
0      29.784623        1       1 -0.037476 -0.127435
18      6.614733        1       2  0.002522 -0.224472
21      5.796221        1       3  0.067543 -0.171179
17      4.173119        1       4 -0.169847 -0.225670
7       3.896502        1       5 -0.115000 -0.045756
9       3.853071        1       6  0.224146 -0.044962
10      3.719569        1       7 -0.176139 -0.096821
22      3.441303        1       8  0.275840 -0.048074
15      3.400347        1       9 -0.044701 -0.173192
11      3.174333        1      10 -0.043310  0.124365
13      3.145613        1      11 -0.074966  0.001203
8       3.078506        1      12 -0.143686  0.136854
4       3.077213        1      13 -0.159010 -0.138386
23      2.728995        1      14 -0.151404 -0.101640
6       2.727949        1      15  0.233379 -0.061315
16      2.645923        1      16  0.197639 -0.054968
19      2.328169        1      17 -0.192593  0.093117
20      2.089112        1      18  0.080238  0.141384
24      2.034363        1      19  0.077271  0.148543
14      1.979094        1      20 -0.185195  0.202764
1       1.466575        1      21  0.272002 -0.002529
5       1.343676        1      22  0.104644  0.184683
3       1.276497        1      23 -0.113905  0.236275
2       1.186444        1      24  0.134218  0.160570
12      1.038050        1      25 -0.062209  0.086640, topic_info=     Category          Freq          Term         Total  loglift  logprob
term                                                                     
2062  Default  10818.000000          game  10818.000000  30.0000  30.0000
2725  Default  11095.000000           kid  11095.000000  29.0000  29.0000
782   Default  12815.000000         child  12815.000000  28.0000  28.0000
1211  Default  10511.000000          data  10511.000000  27.0000  27.0000
5337  Default  11003.000000         woman  11003.000000  26.0000  26.0000
3196  Default   7881.000000         music   7881.000000  25.0000  25.0000
2115  Default   7940.000000          girl   7940.000000  24.0000  24.0000
557   Default   8422.000000         brain   8422.000000  23.0000  23.0000
1305  Default   8720.000000        design   8720.000000  22.0000  22.0000
4135  Default   5968.000000         robot   5968.000000  21.0000  21.0000
815   Default   7588.000000          city   7588.000000  20.0000  20.0000
4497  Default   6514.000000         sound   6514.000000  19.0000  19.0000
722   Default   6888.000000          cell   6888.000000  18.0000  18.0000
4232  Default  10103.000000        school  10103.000000  17.0000  17.0000
658   Default   5313.000000        cancer   5313.000000  16.0000  16.0000
4489  Default   4742.000000          song   4742.000000  15.0000  15.0000
3624  Default   7123.000000          play   7123.000000  14.0000  14.0000
3048  Default   4364.000000           men   4364.000000  13.0000  13.0000
4661  Default   4064.000000       student   4064.000000  12.0000  12.0000
4810  Default   4069.000000       teacher   4069.000000  11.0000  11.0000
1840  Default   4692.000000        family   4692.000000  10.0000  10.0000
3512  Default   4276.000000       patient   4276.000000   9.0000   9.0000
5228  Default   3545.000000         voice   3545.000000   8.0000   8.0000
3620  Default   3553.000000         plant   3553.000000   7.0000   7.0000
5195  Default   3845.000000         video   3845.000000   6.0000   6.0000
1443  Default   3023.000000           dna   3023.000000   5.0000   5.0000
1551  Default   3904.000000     education   3904.000000   4.0000   4.0000
3479  Default   4033.000000        parent   4033.000000   3.0000   3.0000
1306  Default   3111.000000      designer   3111.000000   2.0000   2.0000
375   Default   4303.000000          baby   4303.000000   1.0000   1.0000
...       ...           ...           ...           ...      ...      ...
278   Topic25    114.901394    artificial    236.139396  

### Sum vectorized tokens, convert to dictionary and write out to CSV file for wordcloud visualization

In [22]:
df_counts = pd.DataFrame(tf.toarray(),columns=tfidf_vectorizer.get_feature_names()).T

In [28]:
len(df_counts)

5391

In [24]:
df_counts['total'] = df_counts.sum(axis=1)
df_total = df_counts['total'].sort_values(ascending=False)
counts = dict(df_total)

In [27]:
words = sorted(counts, key=counts.get, reverse=True)
highest = None
lowest = None
for w in words[:100]: # top 100 words
  if highest is None or highest < counts[w] :
    highest = counts[w]
  if lowest is None or lowest > counts[w] :
    lowest = counts[w]

# Spread the font sizes across 20-100 based on the count
bigsize = 80
smallsize = 20

fhand = open('gword.js','w')
fhand.write("gword = [")
first = True
for k in words[:100]:
  if not first : fhand.write( ",\n")
  first = False
  size = counts[k]
  size = (size - lowest) / float(highest - lowest)
  size = int((size * bigsize) + smallsize)
  try:
    fhand.write("{text: '"+k+"', size: "+str(size)+"}")
  except UnicodeEncodeError: continue
fhand.write( "\n];\n")

4

### Count total words

In [48]:
text = list(df.text)

In [49]:
flat = []
for i in text:
    flat.extend(i)

In [52]:
everything = ''.join(flat)

In [55]:
everything = everything.split()

In [56]:
len(everything) # almost 5 million words from the original unprocessed corpus

4759385